In [1]:
import os
import keras as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import LeakyReLU
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam, Adadelta
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from keras import regularizers

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
file = 'dataset_1'

train_datagen = K.preprocessing.image.ImageDataGenerator()

validation_datagen = K.preprocessing.image.ImageDataGenerator()

test_datagen = K.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        os.path.join(file, 'train'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode='categorical',
        )

validation_generator = validation_datagen.flow_from_directory(
        os.path.join(file, 'validation'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode="categorical",
        )

test_generator = validation_datagen.flow_from_directory(
        os.path.join(file, 'test'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode="categorical",
        )

Found 12556 images belonging to 28 classes.
Found 1710 images belonging to 28 classes.
Found 3515 images belonging to 28 classes.


In [3]:
def runGenerator(generator):
    X = []
    y = []
    for i in range(len(generator)):
        X_ex, y_ex = generator.next()
        X.append(X_ex)
        y.append(y_ex)
    return X, y

def F1ScoreCalculator(test_generatorm, model):
    X_test = []
    y_test = []
    y_pred = []
    for i in range(len(test_generator)):
        X_ex, y_ex = test_generator.next()
        X_test.append(X_ex)
        y_test.append(y_ex.argmax(axis=-1))
        y_pred.append(model.predict(X_ex).argmax(axis=-1))

    return f1_score(y_test, y_pred, average='micro')

In [8]:
def Create_model(kernel_sizes=5, neurons_bk1=20, neurons_bk2=50, learn_rate=0.001, optomizer='sgd', activation='relu', stride_1=1, stride_2=1):
    print('kernel_sizes= {} | neurons_bk1= {} | neurons_bk2= {} | learn_rate={} | optomizer={} | activation={} | stride_1={} | stride_2={}'.format(
        kernel_sizes, 
        neurons_bk1, 
        neurons_bk2, 
        learn_rate,
        optomizer,
        activation,
        stride_1,
        stride_2
    ))
    
    model = Sequential()
#  -------------   BK1
    model.add(Conv2D(neurons_bk1, kernel_size=kernel_sizes, padding="same", input_shape=(50, 50, 1), strides=stride_1))
    if(activation == 'selu'):
        model.add(Activation('selu'))
    else:
        model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=stride_2))
#   ------------  BK2
    model.add(Conv2D(neurons_bk2, kernel_size=kernel_sizes, border_mode="same", strides=stride_1))
    if(activation == 'selu'):
        model.add(Activation('selu'))
    else:
        model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=stride_2))
#   ------------  BK3
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation('relu'))
#   ------------  BK4
    model.add(Dense(28))
    model.add(Activation("softmax"))
    
#     model.summary()
    if (optomizer == 'adadelta'):
        model.compile(loss="categorical_crossentropy", optimizer=Adadelta(lr=learn_rate), metrics=["accuracy"])
    else:
        model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=learn_rate), metrics=["accuracy"])
    return model

In [9]:
early_stop = K.callbacks.EarlyStopping(monitor='val_acc',
                              min_delta=0,
                              patience=5,
                              verbose=0, mode='max')

In [10]:

kernel_sizes=[3, 5]
neurons_bk1=[5, 7, 10]
neurons_bk2=[3, 5]
learn_rate=[0.01, 0.005]
optomizer=['adadelta', 'adam']
activation=['selu', 'leakrelu']
strides=[1, 2]



In [15]:
INPUT_SHAPE = (50, 50, 1)
NB_EPOCH = 30
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = SGD()
CLASSES = 28
STEPS = (int)(train_generator.n/BATCH_SIZE)

In [16]:
count = 0
max_f1 = 0
all_f1 = []
for ks in kernel_sizes:
    for n_bk1 in neurons_bk1:
        for n_bk2 in  neurons_bk2:
            for lr in learn_rate:
                for opt in optomizer:
                    for act in activation:
                        for std_1 in strides:
                            for std_2 in strides:
                                count += 1
                                print('{}.'.format(count))
                                model = Create_model(kernel_sizes=ks,
                                                    neurons_bk1=n_bk1,
                                                    neurons_bk2=n_bk2,
                                                    learn_rate=lr,
                                                    optomizer=opt,
                                                    activation=act,
                                                    stride_1=std_1,
                                                    stride_2=std_2
                                                    )
                                history = model.fit_generator(
                                    train_generator,
                                    steps_per_epoch=STEPS,
                                    epochs=NB_EPOCH,
                                    verbose=VERBOSE,
                                    validation_data=validation_generator,
                            #         validation_steps=STEPS,
                                    callbacks = [early_stop]
                                )
                                f1_score_val = F1ScoreCalculator(test_generator, model)
                                all_f1.append(f1_score_val)
                                print ("F1-Score = {}".format(f1_score_val))
                                if (f1_score_val > max_f1):
                                    best_model = model
                                    max_f1 = f1_score_val
                                    printer = '{}. kernel_sizes= {} | neurons_bk1= {} | neurons_bk2= {} | learn_rate={} | optomizer={} | activation={} | stride_1={} | stride_2={}'.format(
                                            count,
                                            ks, 
                                            n_bk1, 
                                            n_bk2, 
                                            lr,
                                            opt,
                                            act,
                                            std_1,
                                            std_2)
                                    print ('Best F1-Score')

                                del model
                                del history
                                K.backend.clear_session()
                                K.backend.get_session()

                                print ('------ END MODEL ------\n\n')

print(printer)

1.
kernel_sizes= 3 | neurons_bk1= 5 | neurons_bk2= 3 | learn_rate=0.01 | optomizer=adadelta | activation=selu | stride_1=1 | stride_2=1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, kernel_size=3, strides=1, padding="same")`


Epoch 1/30
98/98 [==============================] - 5s 56ms/step - loss: 14.6205 - acc: 0.0816 - val_loss: 14.5906 - val_acc: 0.0807
Epoch 2/30
98/98 [==============================] - 5s 48ms/step - loss: 14.4737 - acc: 0.1020 - val_loss: 14.5308 - val_acc: 0.0895
Epoch 3/30
98/98 [==============================] - 5s 48ms/step - loss: 13.3910 - acc: 0.1531 - val_loss: 14.4721 - val_acc: 0.0994
Epoch 4/30
98/98 [==============================] - 5s 48ms/step - loss: 14.6379 - acc: 0.0918 - val_loss: 14.4692 - val_acc: 0.1000
Epoch 5/30
98/98 [==============================] - 5s 48ms/step - loss: 14.8734 - acc: 0.0714 - val_loss: 14.4633 - val_acc: 0.0971
Epoch 6/30
98/98 [==============================] - 5s 48ms/step - loss: 15.1313 - acc: 0.0612 - val_loss: 14.4528 - val_acc: 0.0988
Epoch 7/30
98/98 [==============================] - 5s 48ms/step - loss: 14.4303 - acc: 0.0918 - val_loss: 14.4351 - val_acc: 0.1012
Epoch 8/30
98/98 [==============================] - 5s 48ms/step - lo

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(3, kernel_size=3, strides=2, padding="same")`


Epoch 1/30
98/98 [==============================] - 4s 37ms/step - loss: 15.1320 - acc: 0.0510 - val_loss: 15.2007 - val_acc: 0.0322
Epoch 2/30
98/98 [==============================] - 3s 35ms/step - loss: 14.8749 - acc: 0.0408 - val_loss: 14.9328 - val_acc: 0.0357
Epoch 3/30
98/98 [==============================] - 3s 35ms/step - loss: 14.2418 - acc: 0.0816 - val_loss: 14.9816 - val_acc: 0.0357
Epoch 4/30
98/98 [==============================] - 3s 35ms/step - loss: 15.1943 - acc: 0.0408 - val_loss: 14.8646 - val_acc: 0.0596
Epoch 5/30
98/98 [==============================] - 3s 35ms/step - loss: 14.6783 - acc: 0.0816 - val_loss: 14.8589 - val_acc: 0.0608
Epoch 6/30
98/98 [==============================] - 3s 35ms/step - loss: 15.6304 - acc: 0.0306 - val_loss: 14.8492 - val_acc: 0.0643
Epoch 7/30
98/98 [==============================] - 3s 35ms/step - loss: 15.6949 - acc: 0.0102 - val_loss: 14.8183 - val_acc: 0.0626
Epoch 8/30
98/98 [==============================] - 3s 35ms/step - lo

...

98/98 [==============================] - 3s 34ms/step - loss: 15.1313 - acc: 0.0612 - val_loss: 15.5714 - val_acc: 0.0339
Epoch 4/30
98/98 [==============================] - 3s 34ms/step - loss: 15.6247 - acc: 0.0306 - val_loss: 15.5714 - val_acc: 0.0339
Epoch 5/30
98/98 [==============================] - 3s 35ms/step - loss: 15.7892 - acc: 0.0204 - val_loss: 15.5714 - val_acc: 0.0339
Epoch 6/30
98/98 [==============================] - 3s 34ms/step - loss: 15.4602 - acc: 0.0408 - val_loss: 15.5714 - val_acc: 0.0339
F1-Score = 0.03812233285917496
------ END MODEL ------


365.
kernel_sizes= 5 | neurons_bk1= 10 | neurons_bk2= 5 | learn_rate=0.01 | optomizer=adam | activation=leakrelu | stride_1=1 | stride_2=1
Epoch 1/30
98/98 [==============================] - 6s 62ms/step - loss: 15.3232 - acc: 0.0408 - val_loss: 15.6374 - val_acc: 0.0298
Epoch 2/30
98/98 [==============================] - 5s 55ms/step - loss: 15.6247 - acc: 0.0306 - val_loss: 15.6374 - val_acc: 0.0298
Epoch 3/30
98/98 

Epoch 17/30
98/98 [==============================] - 3s 36ms/step - loss: 14.5549 - acc: 0.0612 - val_loss: 14.2685 - val_acc: 0.0930
Epoch 18/30
98/98 [==============================] - 4s 36ms/step - loss: 14.1586 - acc: 0.0918 - val_loss: 14.2293 - val_acc: 0.0994
Epoch 19/30
98/98 [==============================] - 4s 36ms/step - loss: 14.5157 - acc: 0.0816 - val_loss: 14.2044 - val_acc: 0.1029
Epoch 20/30
98/98 [==============================] - 4s 36ms/step - loss: 14.4124 - acc: 0.0816 - val_loss: 14.1772 - val_acc: 0.1035
Epoch 21/30
98/98 [==============================] - 3s 36ms/step - loss: 14.3597 - acc: 0.0918 - val_loss: 14.1634 - val_acc: 0.1070
Epoch 22/30
98/98 [==============================] - 4s 36ms/step - loss: 14.7324 - acc: 0.0816 - val_loss: 14.1615 - val_acc: 0.1094
Epoch 23/30
98/98 [==============================] - 3s 36ms/step - loss: 13.9480 - acc: 0.1224 - val_loss: 14.1478 - val_acc: 0.1105
Epoch 24/30
98/98 [==============================] - 4s 36ms/s

98/98 [==============================] - 3s 35ms/step - loss: 13.9516 - acc: 0.1122 - val_loss: 13.8180 - val_acc: 0.1199
Epoch 11/30
98/98 [==============================] - 3s 35ms/step - loss: 13.4748 - acc: 0.1429 - val_loss: 13.8056 - val_acc: 0.1216
Epoch 12/30
98/98 [==============================] - 3s 35ms/step - loss: 13.2904 - acc: 0.1531 - val_loss: 13.7942 - val_acc: 0.1269
Epoch 13/30
98/98 [==============================] - 3s 35ms/step - loss: 13.8529 - acc: 0.1224 - val_loss: 13.7847 - val_acc: 0.1281
Epoch 14/30
98/98 [==============================] - 3s 35ms/step - loss: 13.5758 - acc: 0.1327 - val_loss: 13.7769 - val_acc: 0.1269
Epoch 15/30
98/98 [==============================] - 3s 35ms/step - loss: 13.4923 - acc: 0.1224 - val_loss: 13.7613 - val_acc: 0.1304
Epoch 16/30
98/98 [==============================] - 3s 35ms/step - loss: 14.0604 - acc: 0.1122 - val_loss: 13.7570 - val_acc: 0.1304
Epoch 17/30
98/98 [==============================] - 3s 35ms/step - loss: 

98/98 [==============================] - 3s 34ms/step - loss: 14.6886 - acc: 0.0816 - val_loss: 14.9053 - val_acc: 0.0620
Epoch 9/30
98/98 [==============================] - 3s 34ms/step - loss: 15.1389 - acc: 0.0510 - val_loss: 14.9087 - val_acc: 0.0626
Epoch 10/30
98/98 [==============================] - 3s 34ms/step - loss: 14.6939 - acc: 0.0714 - val_loss: 14.9042 - val_acc: 0.0626
Epoch 11/30
98/98 [==============================] - 3s 34ms/step - loss: 15.1547 - acc: 0.0408 - val_loss: 14.8911 - val_acc: 0.0632
Epoch 12/30
98/98 [==============================] - 3s 34ms/step - loss: 15.1256 - acc: 0.0612 - val_loss: 14.8844 - val_acc: 0.0637
Epoch 13/30
98/98 [==============================] - 3s 34ms/step - loss: 15.9536 - acc: 0.0102 - val_loss: 14.8844 - val_acc: 0.0637
Epoch 14/30
98/98 [==============================] - 3s 34ms/step - loss: 14.7012 - acc: 0.0714 - val_loss: 14.8773 - val_acc: 0.0661
Epoch 15/30
98/98 [==============================] - 3s 34ms/step - loss: 1

98/98 [==============================] - 3s 35ms/step - loss: 15.6247 - acc: 0.0306 - val_loss: 15.3640 - val_acc: 0.0468
Epoch 5/30
98/98 [==============================] - 3s 34ms/step - loss: 15.4602 - acc: 0.0408 - val_loss: 15.3640 - val_acc: 0.0468
Epoch 6/30
98/98 [==============================] - 3s 35ms/step - loss: 15.6247 - acc: 0.0306 - val_loss: 15.3640 - val_acc: 0.0468
F1-Score = 0.03613086770981508
------ END MODEL ------


384.
kernel_sizes= 5 | neurons_bk1= 10 | neurons_bk2= 5 | learn_rate=0.005 | optomizer=adam | activation=leakrelu | stride_1=2 | stride_2=2
Epoch 1/30
98/98 [==============================] - 4s 37ms/step - loss: 15.1313 - acc: 0.0612 - val_loss: 15.5431 - val_acc: 0.0357
Epoch 2/30
98/98 [==============================] - 3s 34ms/step - loss: 15.1313 - acc: 0.0612 - val_loss: 15.5431 - val_acc: 0.0357
Epoch 3/30
98/98 [==============================] - 3s 34ms/step - loss: 15.2957 - acc: 0.0510 - val_loss: 15.5431 - val_acc: 0.0357
Epoch 4/30
98/98